In [1]:
from sage.all import *
pretty_print_default(True)
%display latex

In [2]:
Λ, τ, σ = var('Lambda','tau','sigma')
σ2 = σ^2

In [3]:
def delta_contrast_Zel(d):
    """
    The Zeldovich toy-model Δ(τ): one convenient default delta_func.
    """
    taus = get_tau_vars(d)
    T = Matrix(SR, d); k = 0
    for i in range(d):
        T[i,i] = taus[k]; k += 1
    for i in range(d):
        for j in range(i+1,d):
            T[i,j] = T[j,i] = taus[k]; k += 1
    return 1/((identity_matrix(SR,d) - T).det()) - 1

# Ellipsoidal collapse in terms of τ‐variables (Eq. 5.3):
def delta_contrast_ellipsoidal(d, nu_param = var('nu')):
    """
    Ellipsoidal‐collapse density contrast Δ(τ) ≡ ρ(τ) − 1
      - d         : dimension (e.g. 3)
      - nu_param  : ellipticity exponent ν
    """

    taus = get_tau_vars(d)
    T = Matrix(SR, d); k = 0
    for i in range(d):
        T[i,i] = taus[k]; k += 1
    for i in range(d):
        for j in range(i+1,d):
            T[i,j] = T[j,i] = taus[k]; k += 1
    
    # 2) linear contrast δ = sum_i tau_ii
    delta_lin = T.trace()
    
    # 3) build the total density ρ(τ) and subtract 1 to get Δ
    rho = (1 - delta_lin/d)^d / (1 - delta_lin/nu_param)^nu_param / ((identity_matrix(SR,d) - T).det())
    
    return rho - 1


In [4]:
# ──────────────────────────────────────────────────────────────────────────────
# Block 1: Core helper functions (generic saddle‐point CGF framework)
#           with default nu = d
# ──────────────────────────────────────────────────────────────────────────────

def get_tau_vars(d):
    diag = [var(f'tau{i}{i}') for i in range(1, d+1)]
    off  = [var(f'tau{i}{j}') for i in range(1, d+1) for j in range(i+1, d+1)]
    return diag + off

def precision_matrix(d):
    if d==2:
        C = Matrix([[3/8,1/8,0],
                    [1/8,3/8,0],
                    [0,  0,  1/8]])
    elif d==3:
        C = Matrix([[1/5, 1/15,1/15, 0,    0,    0],
                    [1/15,1/5, 1/15, 0,    0,    0],
                    [1/15,1/15,1/5,  0,    0,    0],
                    [0,   0,   0,    1/15, 0,    0],
                    [0,   0,   0,    0,    1/15, 0],
                    [0,   0,   0,    0,    0,    1/15]] )
    else:
        raise ValueError("Only d=2 or 3 supported")
    return C.inverse()

def action(d, Λ, nu=None):
    if nu is None:
        nu = d
    taus = get_tau_vars(d)
    Δ    = delta_contrast_ellipsoidal(d, nu)
    P    = precision_matrix(d)
    quad = sum(taus[i] * sum(P[i,j]*taus[j] for j in range(len(taus)))
               for i in range(len(taus)))
    return Λ*Δ - (1/2)*quad

def Lambda_of_tau(d, nu=None):
    if nu is None:
        nu = d
    Λ, τ = var('Lambda','tau')
    taus = get_tau_vars(d)
    eq = diff(action(d, Λ, nu), taus[0]).subs(
         {**{v:τ for v in taus[:d]},
          **{v:0 for v in taus[d:]}}
    )
    sol = solve(eq, Λ)[0]
    return sol.rhs().factor()

def CGF_LO(d, nu=None):
    if nu is None:
        nu = d
    Λ, τ = var('Lambda','tau')
    Λτ   = Lambda_of_tau(d, nu)
    taus = get_tau_vars(d)
    subs = {v:τ for v in taus[:d]}
    subs.update({v:0 for v in taus[d:]})
    subs[Λ] = Λτ
    return action(d, Λ, nu).subs(subs).factor()

def hessian_full(d, Λ, nu=None):
    if nu is None:
        nu = d
    taus = get_tau_vars(d)
    S    = action(d, Λ, nu)
    n    = len(taus)
    H    = Matrix(SR, n, n)
    for i in range(n):
        for j in range(n):
            H[i,j] = diff(S, taus[i], taus[j])
    return H

def hessian_spherical(d, nu=None):
    if nu is None:
        nu = d
    Λ, τ = var('Lambda','tau')
    Hf   = hessian_full(d, Λ, nu)
    taus = get_tau_vars(d)
    subs = {v:τ for v in taus[:d]}
    subs.update({v:0 for v in taus[d:]})
    subs[Λ] = Lambda_of_tau(d, nu)
    return Hf.subs(subs).apply_map(lambda x: x.factor())

def one_loop_correction(d, nu=None):
    if nu is None:
        nu = d
    Hs = -hessian_spherical(d, nu)
    return - (1/2) * log( (Hs.det()/precision_matrix(d).det()).factor() )

def CGF_NLO(d, σ2, nu=None):
    if nu is None:
        nu = d
    φ0 = CGF_LO(d,      nu)
    F1 = one_loop_correction(d, nu)
    return (φ0 + σ2*F1).factor()

def split_one_loop(d, nu=None):
    if nu is None:
        nu = d
    Hs = -hessian_spherical(d, nu)
    Pm = precision_matrix(d)
    H  = Hs/Pm

    N  = len(get_tau_vars(d))
    evs= H.eigenvalues()
    mults = [(evs.count(e), e) for e in set(evs)]
    λ_iso   = [e for m,e in mults if m==1][0]
    λ_aniso = [e for m,e in mults if m>1][0]

    F1_iso   = -(1/2)*log(λ_iso)
    F1_aniso = -(1/2)*(N-1)*log(λ_aniso)
    return F1_iso, F1_aniso

In [5]:
def driver_show_CGF(d, nu=None):
    """
    show Δ(τ), Λ(τ), φ0(τ), F1(τ), and φ_NLO(τ), all factorized.
    """
    if nu is None:
        nu = d
    tau = var('tau')
    Δ = delta_contrast_ellipsoidal(d, nu)

    # 1) expansion parameter Λ(τ)
    Λ_expr = Lambda_of_tau(d, nu)

    # 2) leading CGF and 1-loop corrections
    φ0_expr  = CGF_LO(d, nu)
    F1_expr  = one_loop_correction(d, nu)
    φNLO_expr = CGF_NLO(d, σ2, nu)

    # display everything, factorized
    show("Δ(τ) =", Δ)
    show("Λ(τ) =", Λ_expr)
    show("φ₀(τ) =", φ0_expr)
    show("F₁(τ) =", F1_expr)
    show("φ(τ) =", φNLO_expr)


In [6]:
# pick your model & dimension
driver_show_CGF(d=3)

'Δ(τ) =' 1/((tau23^2 - (tau22 - 1)*(tau33 - 1))*(tau11 - 1) - (tau13*tau23 - tau12*(tau33 - 1))*tau12 + (tau13*(tau22 - 1) - tau12*tau23)*tau13) - 1

'Λ(τ) =' 3*(tau - 1)^4*tau

'φ₀(τ) =' -3/2*(2*tau^3 - 8*tau^2 + 12*tau - 3)*tau^2

'F₁(τ) =' -1/2*log(1/3125*(7*tau - 5)^5*(5*tau - 1)/(tau - 1)^6)

'φ(τ) =' -3*tau^5 + 12*tau^4 - 18*tau^3 - 1/2*sigma^2*log(1/3125*(7*tau - 5)^5*(5*tau - 1)/(tau - 1)^6) + 9/2*tau^2

In [7]:
# pick your model & dimension
driver_show_CGF(d=2,nu=var('nu'))

'Δ(τ) =' -1/4*(tau11 + tau22 - 2)^2/((tau12^2 - (tau11 - 1)*(tau22 - 1))*(-(tau11 + tau22)/nu + 1)^nu) - 1

'Λ(τ) =' 2*(nu - 2*tau)*tau*((nu - 2*tau)/nu)^nu/nu

'φ₀(τ) =' -2*(nu*tau + nu*((nu - 2*tau)/nu)^nu - 2*tau*((nu - 2*tau)/nu)^nu - nu + 2*tau)*tau/nu

'F₁(τ) =' -1/2*log(-1/4*(2*nu*tau^2 - 5*nu*tau + 2*tau^2 + 2*nu)^2*(2*nu*tau - nu + 4*tau)/((nu - 2*tau)*nu^2*(tau - 1)^4))

'φ(τ) =' -1/2*(nu*sigma^2*log(-1/4*(2*(nu + 1)*tau^2 - 5*nu*tau + 2*nu)^2*(2*(nu + 2)*tau - nu)/((nu - 2*tau)*nu^2*(tau - 1)^4)) + 4*nu*tau^2 + 4*nu*tau*((nu - 2*tau)/nu)^nu - 8*tau^2*((nu - 2*tau)/nu)^nu - 4*nu*tau + 8*tau^2)/nu

In [8]:
# ──────────────────────────────────────────────────────────────────────────────
# Series‐building block, using the unified ellipsoidal action (nu=3 ⇒ Zel’dovich)
# ──────────────────────────────────────────────────────────────────────────────
def series_build(d, N=6, nu=None):
    """
    Returns (tau_of_L,
             phi0_tau, phi0_L,
             phi1_tau, phi1_L,
             phi_tau_total, phi_L_total)
    where
      - phi0_tau = φ₀(τ)
      - phi1_tau = F₁(τ)
      - phi_tau_total = φ₀(τ) + σ² F₁(τ)
      - phi0_L, phi1_L are the PowerSeries in Λ (over QQ) for φ₀ and F₁
      - phi_L_total is the symbolic Λ‐series (LO + σ²·NLO)
    """
    if nu is None:
        nu = d  # default to Zel'dovich
    # 0) declare your symbols
    Lambda, tau, sigma = var('Lambda','tau','sigma')
    σ2 = sigma^2

    # 1) build the core saddle‐point CGFs
    Λ_of_tau  = Lambda_of_tau(d, nu)         # Λ(τ)
    phi0_tau  = CGF_LO(d,      nu)           # φ₀(τ)
    phi1_tau  = one_loop_correction(d, nu)   # F₁(τ)

    # 2) Taylor‐expand these in τ up to order N
    Λ_tay     = taylor(Λ_of_tau,  tau, 0, N)
    phi0_tay  = taylor(phi0_tau,  tau, 0, N)
    phi1_tay  = taylor(phi1_tau,  tau, 0, N)

    # 3) build the power‐series ring in Λ over the Symbolic Ring
    R.<Λ>     = PowerSeriesRing(SR, default_prec=N)

    # 4) invert Λ(τ) → τ(Λ)
    ps_L      = sum(Λ_tay.coefficient(tau,i) * Λ^i for i in range(1, N))
    tau_of_L  = ps_L.reverse(N)

    # 5) build φ₀(Λ) as a PS‐ring element
    phi0_L = R( [ phi0_tay.coefficient(tau,i) for i in range(N) ] )
    # Actually we need to substitute τ→τ(Λ), so:
    phi0_L = R([ (sum(phi0_tay.coefficient(tau,j) * tau_of_L**j for j in range(N)))[i]
                for i in range(N) ])

    # 6) same for F₁(Λ)
    phi1_L = R([ (sum(phi1_tay.coefficient(tau, j) * tau_of_L**j for j in range(N)))[i]
                for i in range(N) ])

    # 7) build the total φ series in τ
    phi_tau_total = phi0_tau + σ2*phi1_tau

    # 8) assemble the symbolic Λ‐series φ(Λ) = φ₀(Λ) + σ²·F₁(Λ)
    coeff0 = phi0_L.list()
    coeff1 = phi1_L.list()
    coeffs_tot = [ coeff0[i] + σ2*coeff1[i] for i in range(N) ]
    phi_L_total = sum( coeffs_tot[i] * Lambda^i for i in range(N) )

    return tau_of_L, phi0_tau, phi0_L, phi1_tau, phi1_L, phi_tau_total, phi_L_total


# ──────────────────────────────────────────────────────────────────────────────
# Minimal driver to check it
# ──────────────────────────────────────────────────────────────────────────────
def driver_series_CGF(d, N=6, nu=None):
    if nu is None:
        nu = d  # default to Zel'dovich
    tau_of_L, φ0_τ, φ0_L, φ1_τ, φ1_L, φ_τ, φ_L = series_build(d, N, nu)

    show("Λ(τ) =", Lambda_of_tau(d, nu))
    show("τ(Λ) =", tau_of_L)
    show("φ₀(τ) =", φ0_τ.factor())
    show("F₁(τ) =", φ1_τ.factor())
    show("φ(τ) =",  φ_τ.factor())

    show("φ₀(Λ) =", φ0_L.truncate(N))
    show("F₁(Λ) =", φ1_L.truncate(N))
    show("φ(Λ) =",  φ_L)


# ──────────────────────────────────────────────────────────────────────────────

In [9]:
# ──────────────────────────────────────────────────────────────────────────────
# Series‐building block, using the unified ellipsoidal action (nu=3 ⇒ Zel'dovich)
# Note: For exact expressions, use driver_analytical_CGF() instead

# ──────────────────────────────────────────────────────────────────────────────
driver_series_CGF(3, 6)  # Example for d=3, N=6
# ──────────────────────────────────────────────────────────────────────────────

'Λ(τ) =' 3*(tau - 1)^4*tau

'τ(Λ) =' 1/3*Λ + 4/9*Λ^2 + 26/27*Λ^3 + 68/27*Λ^4 + 1771/243*Λ^5 + O(Λ^6)

'φ₀(τ) =' -3/2*(2*tau^3 - 8*tau^2 + 12*tau - 3)*tau^2

'F₁(τ) =' -1/2*log(1/3125*(7*tau - 5)^5*(5*tau - 1)/(tau - 1)^6)

'φ(τ) =' -3*tau^5 + 12*tau^4 - 18*tau^3 - 1/2*sigma^2*log(1/3125*(7*tau - 5)^5*(5*tau - 1)/(tau - 1)^6) + 9/2*tau^2

'φ₀(Λ) =' 77/27*Λ^5 + 34/27*Λ^4 + 2/3*Λ^3 + 1/2*Λ^2

'F₁(Λ) =' 4904974/84375*Λ^5 + 60284/3375*Λ^4 + 3943/675*Λ^3 + 32/15*Λ^2 + Λ

'φ(Λ) =' 1/84375*(4904974*sigma^2 + 240625)*Lambda^5 + 2/3375*(30142*sigma^2 + 2125)*Lambda^4 + 1/675*(3943*sigma^2 + 450)*Lambda^3 + 1/30*(64*sigma^2 + 15)*Lambda^2 + Lambda*sigma^2

In [10]:
driver_series_CGF(d=3, N=6, nu=var('nu'))              # keeps nu symbolic

'Λ(τ) =' 3*(nu - 3*tau)*tau*((nu - 3*tau)/nu)^nu/nu

'τ(Λ) =' 1/3*Λ + (1/3*(nu + 1)/nu)*Λ^2 + (2/3*(nu + 1)^2/nu^2 - 1/6*(nu + 1)/nu)*Λ^3 + (1/8*(nu + 1)*(4*(nu + 1)^2/nu^2 - (nu + 1)/nu)/nu + 1/12*(nu + 1)*(3*(nu + 1)^2/nu^2 - (nu + 1)/nu)/nu + 11/24*(nu + 1)*(2*(nu + 1)^2/nu^2 - (nu + 1)/nu)/nu - 1/6*(nu + 1)^2/nu^2 + 1/18*(nu^2 - 1)/nu^2)*Λ^4 + (1/15*(5*(nu + 1)^2/nu^2 - (nu + 1)/nu)*(2*(nu + 1)^2/nu^2 - (nu + 1)/nu) + 1/20*(4*(nu + 1)^2/nu^2 - (nu + 1)/nu)*(2*(nu + 1)^2/nu^2 - (nu + 1)/nu) + 1/30*(3*(nu + 1)^2/nu^2 - (nu + 1)/nu)*(2*(nu + 1)^2/nu^2 - (nu + 1)/nu) + 1/60*(2*(nu + 1)^2/nu^2 - (nu + 1)/nu)^2 + 1/90*(nu + 1)*(9*(nu + 1)*(4*(nu + 1)^2/nu^2 - (nu + 1)/nu)/nu + 6*(nu + 1)*(3*(nu + 1)^2/nu^2 - (nu + 1)/nu)/nu + 33*(nu + 1)*(2*(nu + 1)^2/nu^2 - (nu + 1)/nu)/nu - 12*(nu + 1)^2/nu^2 + 4*(nu^2 - 1)/nu^2)/nu + 1/30*(nu + 1)*(2*(nu + 1)*(3*(nu + 1)^2/nu^2 - (nu + 1)/nu)/nu + 7*(nu + 1)*(2*(nu + 1)^2/nu^2 - (nu + 1)/nu)/nu - 3*(nu + 1)^2/nu^2 + (nu^2 - 1)/nu^2)/nu + 1/45*(nu + 1)*(6*(nu + 1)*(2*(nu + 1)^2/nu^2 - (nu + 1)/nu)/nu - 3*(nu + 1)^2/nu^2 + (nu^2 - 1)/nu^2)/nu + 8/45*(nu + 1)*(3*(nu + 1)*(2*(nu + 1)^2/nu^2 - (nu + 1)/nu)/nu - 3*(nu + 1)^2/nu^2 + (nu^2 - 1)/nu^2)/nu - 1/12*(nu + 1)*(2*(nu + 1)^2/nu^2 - (nu + 1)/nu)/nu + 1/18*(nu^2 - 1)*(nu + 1)/nu^3 - 1/72*(nu^3 - 2*nu^2 - nu + 2)/nu^3)*Λ^5 + O(Λ^6)

'φ₀(τ) =' -3/2*(3*nu*tau + 2*nu*((nu - 3*tau)/nu)^nu - 6*tau*((nu - 3*tau)/nu)^nu - 2*nu + 6*tau)*tau/nu

'F₁(τ) =' -1/2*log(-1/3125*((5*nu + 6)*tau^2 - 12*nu*tau + 5*nu)^5*(3*(nu + 2)*tau - nu)/((nu - 3*tau)*nu^5*(tau - 1)^10))

'φ(τ) =' -1/2*(nu*sigma^2*log(-1/3125*((5*nu + 6)*tau^2 - 12*nu*tau + 5*nu)^5*(3*(nu + 2)*tau - nu)/((nu - 3*tau)*nu^5*(tau - 1)^10)) + 9*nu*tau^2 + 6*nu*tau*((nu - 3*tau)/nu)^nu - 18*tau^2*((nu - 3*tau)/nu)^nu - 6*nu*tau + 18*tau^2)/nu

'φ₀(Λ) =' (-3/4*(nu + 1)*(6*(nu + 1)^2/nu^2 - (nu + 1)/nu)/nu + 7/8*(nu + 1)*(4*(nu + 1)^2/nu^2 - (nu + 1)/nu)/nu + 1/4*(nu + 1)*(3*(nu + 1)^2/nu^2 - (nu + 1)/nu)/nu + 11/8*(nu + 1)*(2*(nu + 1)^2/nu^2 - (nu + 1)/nu)/nu - 1/2*(nu + 1)^2/nu^2 + 2/3*(nu^2 - 1)*(nu + 1)/nu^3 + 1/6*(nu^2 - 1)/nu^2 - 1/24*(nu^3 - 2*nu^2 - nu + 2)/nu^3)*Λ^5 + ((nu + 1)^2/nu^2 - 1/2*(nu + 1)/nu + 1/6*(nu^2 - 1)/nu^2)*Λ^4 + (1/2*(nu + 1)/nu)*Λ^3 + 1/2*Λ^2

'F₁(Λ) =' (1/720*(24*(5*(nu + 1)^2/nu^2 - (nu + 1)/nu)*(2*(nu + 1)^2/nu^2 - (nu + 1)/nu) + 18*(4*(nu + 1)^2/nu^2 - (nu + 1)/nu)*(2*(nu + 1)^2/nu^2 - (nu + 1)/nu) + 12*(3*(nu + 1)^2/nu^2 - (nu + 1)/nu)*(2*(nu + 1)^2/nu^2 - (nu + 1)/nu) + 6*(2*(nu + 1)^2/nu^2 - (nu + 1)/nu)^2 + 4*(nu + 1)*(9*(nu + 1)*(4*(nu + 1)^2/nu^2 - (nu + 1)/nu)/nu + 6*(nu + 1)*(3*(nu + 1)^2/nu^2 - (nu + 1)/nu)/nu + 33*(nu + 1)*(2*(nu + 1)^2/nu^2 - (nu + 1)/nu)/nu - 12*(nu + 1)^2/nu^2 + 4*(nu^2 - 1)/nu^2)/nu + 12*(nu + 1)*(2*(nu + 1)*(3*(nu + 1)^2/nu^2 - (nu + 1)/nu)/nu + 7*(nu + 1)*(2*(nu + 1)^2/nu^2 - (nu + 1)/nu)/nu - 3*(nu + 1)^2/nu^2 + (nu^2 - 1)/nu^2)/nu + 8*(nu + 1)*(6*(nu + 1)*(2*(nu + 1)^2/nu^2 - (nu + 1)/nu)/nu - 3*(nu + 1)^2/nu^2 + (nu^2 - 1)/nu^2)/nu + 64*(nu + 1)*(3*(nu + 1)*(2*(nu + 1)^2/nu^2 - (nu + 1)/nu)/nu - 3*(nu + 1)^2/nu^2 + (nu^2 - 1)/nu^2)/nu - 30*(nu + 1)*(2*(nu + 1)^2/nu^2 - (nu + 1)/nu)/nu + 20*(nu^2 - 1)*(nu + 1)/nu^3 - 5*(nu^3 - 2*nu^2 - nu + 2)/nu^3)*(5*nu + 3)/nu + 1/2160*(89*nu^2 + 120*nu + 135)*(21*(nu + 1)*(4*(nu + 1)^2/nu^2 - (nu + 1)/nu)/nu + 6*(nu + 1)*(3*(nu + 1)^2/nu^2 - (nu + 1)/nu)/nu + 33*(nu + 1)*(2*(nu + 1)^2/nu^2 - (nu + 1)/nu)/nu - 12*(nu + 1)^2/nu^2 + 4*(nu^2 - 1)/nu^2)/nu^2 + 1/2700*(1253*nu^3 + 2970*nu^2 + 8100*nu + 4725)*(6*(nu + 1)^2/nu^2 - (nu + 1)/nu)/nu^3 + 1/6750*(5487*nu^4 + 22240*nu^3 + 81600*nu^2 + 108000*nu + 50625)*(nu + 1)/nu^5 + 1/1518750*(215957*nu^5 + 1349550*nu^4 + 6129000*nu^3 + 12150000*nu^2 + 12150000*nu + 4708125)/nu^5)*Λ^5 + (1/144*(5*nu + 3)*(9*(nu + 1)*(4*(nu + 1)^2/nu^2 - (nu + 1)/nu)/nu + 6*(nu + 1)*(3*(nu + 1)^2/nu^2 - (nu + 1)/nu)/nu + 33*(nu + 1)*(2*(nu + 1)^2/nu^2 - (nu + 1)/nu)/nu - 12*(nu + 1)^2/nu^2 + 4*(nu^2 - 1)/nu^2)/nu + 1/180*(89*nu^2 + 120*nu + 135)*(5*(nu + 1)^2/nu^2 - (nu + 1)/nu)/nu^2 + 1/1350*(1253*nu^3 + 2970*nu^2 + 8100*nu + 4725)*(nu + 1)/nu^4 + 1/27000*(5487*nu^4 + 22240*nu^3 + 81600*nu^2 + 108000*nu + 50625)/nu^4)*Λ^4 + (1/12*(5*nu + 3)*(4*(nu + 1)^2/nu^2 - (nu + 1)/nu)/nu + 1/90*(89*nu^2 + 120*nu + 135)*(nu + 1)/nu^3 + 1/4050*(1253*nu^3 + 2970*nu^2 + 8100*nu + 4725)/nu^3)*Λ^3 + (1/6*(5*nu + 3)*(nu + 1)/nu^2 + 1/180*(89*nu^2 + 120*nu + 135)/nu^2)*Λ^2 + (1/6*(5*nu + 3)/nu)*Λ

'φ(Λ) =' 1/12150000*((16875*(24*(5*(nu + 1)^2/nu^2 - (nu + 1)/nu)*(2*(nu + 1)^2/nu^2 - (nu + 1)/nu) + 18*(4*(nu + 1)^2/nu^2 - (nu + 1)/nu)*(2*(nu + 1)^2/nu^2 - (nu + 1)/nu) + 12*(3*(nu + 1)^2/nu^2 - (nu + 1)/nu)*(2*(nu + 1)^2/nu^2 - (nu + 1)/nu) + 6*(2*(nu + 1)^2/nu^2 - (nu + 1)/nu)^2 + 4*(nu + 1)*(9*(nu + 1)*(4*(nu + 1)^2/nu^2 - (nu + 1)/nu)/nu + 6*(nu + 1)*(3*(nu + 1)^2/nu^2 - (nu + 1)/nu)/nu + 33*(nu + 1)*(2*(nu + 1)^2/nu^2 - (nu + 1)/nu)/nu - 12*(nu + 1)^2/nu^2 + 4*(nu^2 - 1)/nu^2)/nu + 12*(nu + 1)*(2*(nu + 1)*(3*(nu + 1)^2/nu^2 - (nu + 1)/nu)/nu + 7*(nu + 1)*(2*(nu + 1)^2/nu^2 - (nu + 1)/nu)/nu - 3*(nu + 1)^2/nu^2 + (nu^2 - 1)/nu^2)/nu + 8*(nu + 1)*(6*(nu + 1)*(2*(nu + 1)^2/nu^2 - (nu + 1)/nu)/nu - 3*(nu + 1)^2/nu^2 + (nu^2 - 1)/nu^2)/nu + 64*(nu + 1)*(3*(nu + 1)*(2*(nu + 1)^2/nu^2 - (nu + 1)/nu)/nu - 3*(nu + 1)^2/nu^2 + (nu^2 - 1)/nu^2)/nu - 30*(nu + 1)*(2*(nu + 1)^2/nu^2 - (nu + 1)/nu)/nu + 20*(nu^2 - 1)*(nu + 1)/nu^3 - 5*(nu^3 - 2*nu^2 - nu + 2)/nu^3)*(5*nu + 3)/nu + 5625*(89*nu^2 + 120*nu + 135)*(21*(nu + 1)*(4*(nu + 1)^2/nu^2 - (nu + 1)/nu)/nu + 6*(nu + 1)*(3*(nu + 1)^2/nu^2 - (nu + 1)/nu)/nu + 33*(nu + 1)*(2*(nu + 1)^2/nu^2 - (nu + 1)/nu)/nu - 12*(nu + 1)^2/nu^2 + 4*(nu^2 - 1)/nu^2)/nu^2 + 4500*(1253*nu^3 + 2970*nu^2 + 8100*nu + 4725)*(6*(nu + 1)^2/nu^2 - (nu + 1)/nu)/nu^3 + 1800*(5487*nu^4 + 22240*nu^3 + 81600*nu^2 + 108000*nu + 50625)*(nu + 1)/nu^5 + 8*(215957*nu^5 + 1349550*nu^4 + 6129000*nu^3 + 12150000*nu^2 + 12150000*nu + 4708125)/nu^5)*sigma^2 - 9112500*(nu + 1)*(6*(nu + 1)^2/nu^2 - (nu + 1)/nu)/nu + 10631250*(nu + 1)*(4*(nu + 1)^2/nu^2 - (nu + 1)/nu)/nu + 3037500*(nu + 1)*(3*(nu + 1)^2/nu^2 - (nu + 1)/nu)/nu + 16706250*(nu + 1)*(2*(nu + 1)^2/nu^2 - (nu + 1)/nu)/nu - 6075000*(nu + 1)^2/nu^2 + 8100000*(nu^2 - 1)*(nu + 1)/nu^3 + 2025000*(nu^2 - 1)/nu^2 - 506250*(nu^3 - 2*nu^2 - nu + 2)/nu^3)*Lambda^5 + 1/54000*(sigma^2*(375*(5*nu + 3)*(9*(nu + 1)*(4*(nu + 1)^2/nu^2 - (nu + 1)/nu)/nu + 6*(nu + 1)*(3*(nu + 1)^2/nu^2 - (nu + 1)/nu)/nu + 33*(nu + 1)*(2*(nu + 1)^2/nu^2 - (nu + 1)/nu)/nu - 12*(nu + 1)^2/nu^2 + 4*(nu^2 - 1)/nu^2)/nu + 300*(89*nu^2 + 120*nu + 135)*(5*(nu + 1)^2/nu^2 - (nu + 1)/nu)/nu^2 + 40*(1253*nu^3 + 2970*nu^2 + 8100*nu + 4725)*(nu + 1)/nu^4 + 2*(5487*nu^4 + 22240*nu^3 + 81600*nu^2 + 108000*nu + 50625)/nu^4) + 54000*(nu + 1)^2/nu^2 - 27000*(nu + 1)/nu + 9000*(nu^2 - 1)/nu^2)*Lambda^4 + 1/8100*(sigma^2*(675*(5*nu + 3)*(4*(nu + 1)^2/nu^2 - (nu + 1)/nu)/nu + 90*(89*nu^2 + 120*nu + 135)*(nu + 1)/nu^3 + 2*(1253*nu^3 + 2970*nu^2 + 8100*nu + 4725)/nu^3) + 4050*(nu + 1)/nu)*Lambda^3 + 1/180*(sigma^2*(30*(5*nu + 3)*(nu + 1)/nu^2 + (89*nu^2 + 120*nu + 135)/nu^2) + 90)*Lambda^2 + 1/6*Lambda*(5*nu + 3)*sigma^2/nu

In [11]:
# 3) Simplified density cumulants driver
def driver_density_cumulants(d, N, nu=None):
    """
    Compute κ_n^(δ) and κ_n^(ρ) as plain lists (symbolic).
    """
    if nu is None:
        nu = d
    # get the raw phi series in τ
    _, _, φ0_L, _, φ1_L, _, _ = series_build(d, N, nu)

    # extract coefficients bₙ
    Λ, τ, σ = var('Lambda','tau','sigma')
    σ2 = σ^2

    b0 = φ0_L.list()
    b1 = φ1_L.list()
    
    b1_new = [SR(b1[i])*σ*σ for i in range(len(b1))]
    #show(b1_new)
    
    coeffs_tot = [SR(b0[i]) + b1_new[i] for i in range(len(b0))]
    #show(coeffs_tot)

    # 2) density mean μ = 1 + σ²  ⇒  density cumulants κₙ^(ρ) = μⁿ · κₙ^(δ)
    kappa_rho = {}
    kappa_rho[1] = coeffs_tot[1] + 1
    for n in range(2,N):
        kappa_rho[n] = coeffs_tot[n]
    #show(kappa_rho)

    k_rho = {}
    for n in range(1, N):
    # multiply out and truncate at O(σ²)
        expr = kappa_rho[n] / kappa_rho[1]**n *factorial(n)
        k_rho[n] = expr.series(σ, 4)
        #show(f"κ_{n}^(ρ) =", kappa_rho[n])
    #show(k_rho)
    # 3) (Optional) reduced cumulants Sₙ = κₙ^(ρ) / [κ₂^(ρ)]^(n−1)
    S = {}
    for n in range(3, N):
        S[n] = (k_rho[n] / (k_rho[2]**(n-1))).series(σ,4)
        show(f"S_{n} = ", S[n])

In [12]:
driver_density_cumulants(d=3, N=7)

'S_3 = ' 4 + 1106/225*sigma^2 + Order(sigma^4)

'S_4 = ' 272/9 + 115072/1125*sigma^2 + Order(sigma^4)

'S_5 = ' 3080/9 + 36484376/16875*sigma^2 + Order(sigma^4)

'S_6 = ' 5200 + 2615385152/50625*sigma^2 + Order(sigma^4)

In [13]:
driver_density_cumulants(d=3, N=7,nu=var('nu'))

'S_3 = ' (3/nu + 3) + (-1/15*(270/nu + 225/nu^2 + 89)*(1/nu + 1) + 85/3/nu + 45/nu^2 + 22/nu^3 + 5258/675)*sigma^2 + Order(sigma^4)

'S_4 = ' (36/nu + 20/nu^2 + 16) + (-2/15*(270/nu + 225/nu^2 + 89)*(9/nu + 5/nu^2 + 4) + 15554/45/nu + 2318/3/nu^2 + 774/nu^3 + 279/nu^4 + 83947/1125)*sigma^2 + Order(sigma^4)

'S_5 = ' (450/nu + 535/nu^2 + 210/nu^3 + 125) + (-2/9*(270/nu + 225/nu^2 + 89)*(90/nu + 107/nu^2 + 42/nu^3 + 25) + 77282/15/nu + 44048/3/nu^2 + 65650/3/nu^3 + 16120/nu^4 + 4632/nu^5 + 44929628/50625)*sigma^2 + Order(sigma^4)

'S_6 = ' (6480/nu + 12060/nu^2 + 9900/nu^3 + 3024/nu^4 + 1296) + (-2*(270/nu + 225/nu^2 + 89)*(180/nu + 335/nu^2 + 275/nu^3 + 84/nu^4 + 36) + 20321884/225/nu + 4701142/15/nu^2 + 1841510/3/nu^3 + 680570/nu^4 + 397650/nu^5 + 95160/nu^6 + 1938190556/151875)*sigma^2 + Order(sigma^4)

In [14]:
driver_density_cumulants(d=3, N=7,nu=21/13)

'S_3 = ' 34/7 + 17072/1575*sigma^2 + Order(sigma^4)

'S_4 = ' 20264/441 + 769517072/2701125*sigma^2 + Order(sigma^4)

'S_5 = ' 41480/63 + 2173888594432/283618125*sigma^2 + Order(sigma^4)

'S_6 = ' 4363560/343 + 1392987348537248/5955980625*sigma^2 + Order(sigma^4)

In [15]:
def driver_one_loop_split(d,nu=None):
    """
    Split the 1-loop CGF into isotropic & anisotropic pieces,
    display them, and verify they sum back to the full correction.
    
    INPUTS:
      - d               : dimension
      - split_one_loop  : a function with signature
            (d, delta_func) -> (F1_iso(τ), F1_aniso(τ))
      - delta_func      : your density-contrast function Δ(d, τ)
    """
    if nu is None:
        nu = d
    Λ, τ, σ = var('Lambda','tau','sigma')
    σ2 = σ^2

    # 1) get the two pieces
    F1_iso, F1_aniso = split_one_loop(d, nu)

    # 2) show each, multiplied by σ²
    show("Isotropic one–loop F1_iso:",   (σ2*F1_iso).factor())
    show("Anisotropic one–loop F1_aniso:",(σ2*F1_aniso).factor())

    # 3) build their sum exactly as you had in CGF_Zel
    combined = -1/2 * σ2 * log( (exp(-2*F1_iso - 2*F1_aniso)).factor() )
    show("Sum F1_iso + F1_aniso =", combined, 
         "should equal", (one_loop_correction(d, nu)*σ2).factor())

    # return the three pieces for further programmatic use
    #return σ2*F1_iso, σ2*F1_aniso, combined

In [16]:
driver_one_loop_split(3)

'Isotropic one–loop F1_iso:' -1/2*sigma^2*log((5*tau - 1)/(tau - 1))

'Anisotropic one–loop F1_aniso:' -5/2*sigma^2*log(1/5*(7*tau - 5)/(tau - 1))

'Sum F1_iso + F1_aniso =' -1/2*sigma^2*log(1/3125*(7*tau - 5)^5*(5*tau - 1)/(tau - 1)^6) 'should equal' -1/2*sigma^2*log(1/3125*(7*tau - 5)^5*(5*tau - 1)/(tau - 1)^6)

In [17]:
driver_one_loop_split(3,nu=var('nu'))

'Isotropic one–loop F1_iso:' -1/2*sigma^2*log(-(3*(nu + 2)*tau - nu)/(nu - 3*tau))

'Anisotropic one–loop F1_aniso:' -5/2*sigma^2*log(1/5*((5*nu + 6)*tau^2 - 12*nu*tau + 5*nu)/((tau^2 - 2*tau + 1)*nu))

'Sum F1_iso + F1_aniso =' -1/2*sigma^2*log(-1/3125*(5*nu*tau^2 - 12*nu*tau + 6*tau^2 + 5*nu)^5*(3*nu*tau - nu + 6*tau)/((nu - 3*tau)*nu^5*(tau - 1)^10)) 'should equal' -1/2*sigma^2*log(-1/3125*((5*nu + 6)*tau^2 - 12*nu*tau + 5*nu)^5*(3*(nu + 2)*tau - nu)/((nu - 3*tau)*nu^5*(tau - 1)^10))

In [18]:
def compute_tau_critical(d, nu=None):
    """
    Solve d/dτ [Λ(τ)] = 0 and return the largest real critical τ as a float.
    Falls back to 0.25 if no real roots are found.
    """
    if nu is None:
        nu = d
    tau, Lambda = var('tau','Lambda')
    Λ_expr = Lambda_of_tau(d, nu).full_simplify()

    # 1) solve for tau-critical
    sol_eqs = solve(diff(Λ_expr, tau) == 0, tau)
    # sol_eqs is like [tau == a, tau == b, tau == c, ...]

    # 2) extract the right‐hand values
    sol_vals = [ eq.rhs() for eq in sol_eqs ]

    # 4) pick the one you want
    return min(sol_vals)

In [19]:
show('τ_critical:', compute_tau_critical(d=3))
show('τ_critical:',compute_tau_critical(d=3, nu=var('nu')))

'τ_critical:' 1/5

'τ_critical:' 1/6*(nu*((nu - 3*tau)/nu)^(nu - 1) + 2*((nu - 3*tau)/nu)^nu - sqrt(nu^2*((nu - 3*tau)/nu)^(2*nu - 2) + 4*((nu - 3*tau)/nu)^(2*nu)))*((nu - 3*tau)/nu)^(-nu + 1)

In [20]:
import ipywidgets as widgets
# ──────────────────────────────────────────────────────────────────────────────
# 2′) Stable‐axes plotting driver with interactive nu slider
# ──────────────────────────────────────────────────────────────────────────────
@interact
def driver_plot_CGF_interactive(
    d=widgets.Dropdown(options=[2,3], value=3, description='dim d'),
    sigma2=widgets.FloatSlider(min=0.0, max=1.0, step=0.05, value=0.2, description='σ²'),
    nu=widgets.FloatSlider(min=1.0, max=10.0, step=0.1, value=3.0, description='ν'),
    #tau_min=widgets.FloatSlider(min=-1.5, max=0.0, step=0.1, value=-0.9, description='τₘᵢₙ'),
    #pts=widgets.IntSlider(min=100, max=1000, step=100, value=400, description='pts')
):
    """
    Interactive CGF plot with sliders for:
      • d      : dimension (2 or 3)
      • σ²     : variance parameter
      • ν      : ellipticity exponent (ν=3 recovers Zel’dovich)
      • τₘᵢₙ   : lower bound for τ
      • pts    : number of sample points
    """
    tau_min = -1.5
    pts    = 400
    # 1) find tau_cr as pure Python float
    tau_cr_sym = compute_tau_critical(d, nu)
    tau_cr     = float(tau_cr_sym)

    # 2) define symbols
    tau, Lambda = var('tau','Lambda')

    # 3) core expressions
    Λ_of_tau    = Lambda_of_tau(d, nu).full_simplify()
    phi_LO_tau  = CGF_LO(d, nu) + Λ_of_tau
    F1_tau      = one_loop_correction(d, nu)
    phi_NLO_tau = sigma2 * F1_tau

    # 4) iso/aniso split
    F1_iso, F1_aniso = split_one_loop(d, nu)
    phi_iso_tau   = sigma2 * F1_iso
    phi_aniso_tau = sigma2 * F1_aniso

    # 5) build the five curves
    curves = []
    labels = ['LO: $\\phi_0$', 'NLO: $\\phi_1$', 'iso part', 'aniso part', 'iso+aniso']
    style = [
      {'thickness':3,'color':'black'},
      {'thickness':2,'linestyle':'-','color':'red'},
      {'linestyle':'--','thickness':2,'color':'orange'},
      {'linestyle':'--','thickness':2,'color':'blue'},
      {'linestyle':':','thickness':2,'color':'green'}
    ]
    funcs = [phi_LO_tau, phi_NLO_tau, phi_iso_tau, phi_aniso_tau, phi_iso_tau+phi_aniso_tau]

    for label, sty, f in zip(labels, style, funcs):
        p = parametric_plot(
            (Λ_of_tau, f),
            (tau, tau_min, tau_cr),
            plot_points=pts,
            legend_label=label,
            **sty
        )
        curves.append(p)

    G = sum(curves)
    G.set_axes_range(-1, 0.4, -0.5, 0.5)
    G.axes_labels([r'$\Lambda$', r'$\phi_\rho(\Lambda)$'])
    G.legend()

    # 7) show the plot
    show(
        G,
        figsize=[9,6],
        title=f"CGF (d={d}, σ²={sigma2:.2f}, ν={nu:.1f})",
        gridlines=True
    )


Interactive function <function driver_plot_CGF_interactive at 0x7ecc02b5ac00> with 3 widgets
  d: Dropdown(description='dim d', index=1, options=(2, 3), value=3)
  sigma2: FloatSlider(value=0.2, description='σ²', max=1.0, step=0.05)
  nu: FloatSlider(value=3.0, description='ν', max=10.0, min=1.0)

# Inverse Laplace Transform: PDF via Bromwich Integral

Implement raw Bromwich and once-integrated-by-parts formulas to compute the PDF $P(\delta)$ from the cumulant-generating function $\phi(\Lambda)$.

In [28]:
import mpmath as mp
from sage.all import var, diff, fast_callable
# Build numeric callables from symbolic CGF
Λ = var('Lambda')
d=2
nu = d  # Set nu to d for Zeldovich model
sigma_val = 0.2  # Set a numeric value for sigma

# Get the series expansion in Lambda instead of the tau expression
_, _, _, _, _, _, phi_L_total = series_build(d, N=10, nu=nu)
# Substitute the numeric value for sigma
phi_sym = phi_L_total.subs(sigma=sigma_val)
dphi_sym = diff(phi_sym, Λ)

# Create numeric functions using Python's complex numbers
phi = fast_callable(phi_sym, vars=[Λ])
dphi = fast_callable(dphi_sym, vars=[Λ])

def pdf_bromwich(delta_val, sigma0=1.0):
    def integrand(t):
        z = complex(sigma0, t)
        phi_val = complex(phi(z))
        return mp.exp(phi_val - z*delta_val).real
    return (1/(2*mp.pi)) * mp.quad(integrand, [-mp.inf, mp.inf])

def pdf_bromwich_1pi(delta_val, sigma0=1.0):
    def integrand(t):
        z = complex(sigma0, t)
        phi_val = complex(phi(z))
        dphi_val = complex(dphi(z))
        return ((dphi_val - delta_val) * mp.exp(phi_val - z*delta_val)).real
    
    if abs(delta_val) < 1e-10:  # Handle delta_val ≈ 0
        return float('nan')  # or use the raw Bromwich method instead
    
    return (1/(2*mp.pi*delta_val)) * mp.quad(integrand, [-mp.inf, mp.inf])

# Example usage: compute P(δ) for sample values
for δ in [-1.0, -0.5, 0.0, 0.5, 1.0]:
    p_raw = float(pdf_bromwich(δ))
    if abs(δ) < 1e-10:
        p_1pi = p_raw  # Use raw method for delta ≈ 0
    else:
        p_1pi = float(pdf_bromwich_1pi(δ))
    print(f"delta={δ:5.2f}  P(raw)={p_raw:.6g}  P(1×PI)={p_1pi:.6g}")

delta=-1.00  P(raw)=-inf  P(1×PI)=inf
delta=-0.50  P(raw)=-inf  P(1×PI)=inf
delta= 0.00  P(raw)=-inf  P(1×PI)=-inf
delta= 0.50  P(raw)=-inf  P(1×PI)=-inf
delta= 1.00  P(raw)=-inf  P(1×PI)=-inf


# Tau-space Inverse Laplace Transform

Rewrite the Bromwich inversion as an integral in τ, using the analytic mapping Λ(τ) and full action S(τ):

$$
P(\delta) = \frac{1}{2\pi i} \int_{\mathcal C_\tau} d\tau\; \frac{d\Lambda}{d\tau}\, e^{S(\tau)-\Lambda(\tau)\,\delta}.
$$
You can also evaluate the leading saddle-point approximation directly in τ:

$$
P(\delta) \simeq \sqrt{\frac{\Lambda''(\tau^*)}{2\pi}} e^{S(\tau^*)-\delta\,\tau^*},
$$
with Λ(τ*)=δ.

In [52]:
def pdf_tau_saddle(delta_val, d=3, sigma2_val=0.2, nu=None):
    """
    Saddle-point PDF using τ-space formulation:
    P(δ) ≈ √[Λ'(τ*) / 2π] × exp[S(τ*) - δ×τ*]
    where Λ(τ*) = δ
    """
    if nu is None:
        nu = d
    
    # Get symbolic expressions
    tau = var('tau')
    Lambda_tau = Lambda_of_tau(d, nu)
    
    # Build action S(τ) in spherical case
    taus = get_tau_vars(d)
    subs_dict = {v: tau for v in taus[:d]}  # diagonal terms = τ
    subs_dict.update({v: 0 for v in taus[d:]})  # off-diagonal = 0
    subs_dict[Λ] = Lambda_tau  # Add Lambda substitution to the dict
    
    # Total action: S(τ) = action + σ²×F₁(τ)
    S_LO = action(d, Λ, nu).subs(subs_dict)
    F1_tau = one_loop_correction(d, nu)
    S_total = S_LO + sigma2_val * F1_tau
    
    # Derivatives
    dLambda_dtau = diff(Lambda_tau, tau)
    
    # Create numerical functions
    Lam_func = fast_callable(Lambda_tau, vars=[tau])
    S_func = fast_callable(S_total, vars=[tau])  
    dLam_func = fast_callable(dLambda_dtau, vars=[tau])
    
    # Find τ* such that Λ(τ*) = δ
    try:
        # Try different starting points for robustness
        tau_star = None
        for start in [0.0, -0.1, -0.5, -0.8]:
            try:
                tau_star = find_root(lambda t: float(Lam_func(t)) - delta_val, 
                                   start - 0.5, start + 0.5)
                break
            except:
                continue
        
        if tau_star is None:
            return float('nan')
            
        # Saddle-point formula
        prefactor = sqrt(abs(dLam_func(tau_star)) / (2*pi))
        exponent = S_func(tau_star) - delta_val * tau_star
        
        return float(prefactor * exp(exponent))
        
    except Exception as e:
        print(f"Error for δ={delta_val}: {e}")
        return float('nan')

# Test the function
print("Tau-space saddle-point PDF:")
for δ in [-1.0, -0.5, 0.0, 0.5]:
    pdf_val = pdf_tau_saddle(δ, d=3, sigma2_val=0.2, nu=3)
    print(f"δ = {δ:5.2f}  →  P(δ) = {pdf_val:.6g}")

Tau-space saddle-point PDF:
δ = -1.00  →  P(δ) = 1.19909
δ = -1.00  →  P(δ) = 1.19909
δ = -0.50  →  P(δ) = 0.978092
δ = -0.50  →  P(δ) = 0.978092
δ =  0.00  →  P(δ) = 0.690988
δ =  0.00  →  P(δ) = 0.690988
δ =  0.50  →  P(δ) = nan
δ =  0.50  →  P(δ) = nan


In [53]:
import mpmath as mp

# ─── Assume these already exist from your previous cell ────────────────
#   Lambda_num(τ)        = Λ(τ) as a Python function of real/complex τ
#   phi_num(τ)           = φ_τ(τ) as a Python function of real/complex τ
#   dLambda_dtau_num(τ)  = (dΛ/dτ)(τ) as a Python function of real/complex τ
#   tau_c                = the real critical τ (where dΛ/dτ = 0)
tau_c = compute_tau_critical(d)  # Assume this function exists to compute τ_c
# (Re‐define integrand_tau here just to be self‐contained:)
def integrand_tau(tau_val, delta):
    """
    For a real τ ≥ τ_c and real δ, returns
        Λ'(τ) * exp[ φ_τ(τ) - δ⋅Λ(τ) ]  (complex).
    """
    return dLambda_dtau_num(tau_val) * mp.e**( phi_num(tau_val) - delta * Lambda_num(tau_val) )

# ─── (A) Simple real‐axis implementation ────────────────────────────────
def P_of_delta(delta, tau_c):
    """
    Numerically compute
      P(δ) = (1/π) ∫_{τ_c}^{∞}  Im{ Λ'(τ) ⋅ e^{φ_τ(τ) - δ⋅Λ(τ)} }  dτ
    using mpmath.quad.  Returns a real number.
    """
    # Define the integrand for mpmath.quad: the imaginary part of integrand_tau
    def real_axis_integrand(t):
        return mp.im( integrand_tau(t, delta) )
    
    # Perform the integral from τ_c to +∞
    #   You can adjust error tolerances with “epsabs”/“epsrel” if needed.
    I = mp.quad(real_axis_integrand, [tau_c, mp.inf])
    return I / mp.pi

# ─── (B) Example usage ───────────────────────────────────────────────────
delta_example = 0.5
#tau_c = 1.2345    # ← replace with whichever critical τ you computed
P_val = P_of_delta(delta_example, tau_c)
print(f"P(δ={delta_example:g}) ≃ {P_val:g}")


NameError: name 'dLambda_dtau_num' is not defined

In [ ]:
import mpmath as mp
from sage.all import var, diff, fast_callable

# Symbols and analytic functions
tau = var('tau')
Lambda_of_tau_sym = Lambda_of_tau(d, nu)

# For the action, we need to substitute the spherical case (all off-diagonal taus = 0)
taus = get_tau_vars(d)
subs_dict = {v: tau for v in taus[:d]}  # diagonal terms = tau
subs_dict.update({v: 0 for v in taus[d:]})  # off-diagonal terms = 0

# First get the action with the spherical substitution, then substitute Lambda
S_tau_intermediate = action(d, Λ, nu).subs(subs_dict)
S_tau_sym = S_tau_intermediate.subs({Λ: Lambda_of_tau_sym})
dLam_sym = diff(Lambda_of_tau_sym, tau)

# Numeric callables via fast_callable
Lam = fast_callable(Lambda_of_tau_sym, vars=[tau])
Sfun = fast_callable(S_tau_sym, vars=[tau])
dLam_fun = fast_callable(dLam_sym, vars=[tau])

# Saddle-point PDF (one-loop accurate)
def p_saddle(delta_val):
    # find tau* such that Lambda(tau*) = delta
    try:
        # Try multiple starting points and increase tolerance
        for start_point in [0.0, -0.1, -0.5, -0.8]:
            try:
                tau_star = mp.findroot(lambda t: Lam(t) - delta_val, start_point, 
                                     tol=1e-10, maxsteps=100)
                break
            except ValueError:
                continue
        else:
            # If all starting points fail, return NaN
            return float('nan')
        
        pref = mp.sqrt(abs(dLam_fun(tau_star))/(2*mp.pi))
        return pref * mp.e**(Sfun(tau_star) - delta_val*tau_star)
    except (ValueError, ZeroDivisionError):
        return float('nan')

# Exact numeric tau-space integral
def P_numeric(delta_val, line_re=0.0):
    integrand = lambda t: (
        dLam_fun(line_re+1j*t) *
        mp.e**(Sfun(line_re+1j*t) - Lam(line_re+1j*t)*delta_val)
    ).real
    return (1/(2*mp.pi)) * mp.quad(integrand, [-mp.inf, mp.inf])

# Example comparison
for δ in [-1.0, -0.5, 0.0, 0.5, 1.0]:
    saddle_val = float(p_saddle(δ))
    numeric_val = float(P_numeric(δ))
    print(f"delta={δ:5.2f}  saddle={saddle_val:.6g}  numeric={numeric_val:.6g}")

delta=-1.00  saddle=0.985465  numeric=inf
delta=-0.50  saddle=0.835943  numeric=inf
delta= 0.00  saddle=0.56419  numeric=-inf
delta= 0.50  saddle=nan  numeric=inf
delta= 1.00  saddle=nan  numeric=inf


In [ ]:
# ---------------------------------------------------------------------------
#  SADDLE‑POINT PDF  P(δ)  USING τ–VARIABLES  (LO + one‑loop in φ)
# ---------------------------------------------------------------------------
#  Requires the helper functions you already have:
#    • action_spherical_joint(d, Λ, μ, nu)   ––> to build S(τ)
#    • Lambda_of_tau(d, nu)                   ––> analytic Λ(τ)
#    • one_loop_correction(d, nu)             ––> F₁(τ)
#  CGF in τ:  φ(τ) = S(τ)  +  σ²·F₁(τ)
# ---------------------------------------------------------------------------

from sage.all import var, diff, find_root, sqrt, pi, fast_callable, exp

# global symbols
Λ_sym, tau_sym, sigma = var('Lambda tau sigma')
σ2_sym = sigma**2

# ---------------------------------------------------------------------------
# build fast numerical callables for Λ(τ), φ(τ), Λ'(τ)
# ---------------------------------------------------------------------------

def make_tau_callables(d, sigma2_value, nu=None):
    """Return numerical callables Λ(τ), Λ'(τ), φ(τ) for given d,nu,σ²."""
    if nu is None:
        nu = d
    # analytic Λ(τ) and φ(τ) (LO+σ²·F₁)
    Lambda_tau = Lambda_of_tau(d, nu)                    # symbolic
    phi_tau    = (CGF_LO(d, nu) + sigma2_value*one_loop_correction(d, nu))

    # numerical callables (mpmath backend = high precision complex)
    Lam  = fast_callable(Lambda_tau, vars=[tau_sym], domain=ComplexField(50))
    dLam = fast_callable(diff(Lambda_tau, tau_sym), vars=[tau_sym],
                         domain=ComplexField(50))
    phi  = fast_callable(phi_tau,  vars=[tau_sym], domain=ComplexField(50))
    return Lam, dLam, phi

# ---------------------------------------------------------------------------
# saddle‑point PDF function
# ---------------------------------------------------------------------------

def pdf_saddle(delta, d=3, sigma2=1.0, nu=None, tau_left=None, tau_right=None):
    """
    Large‑deviation (LO+1‑loop) PDF for a single density contrast value δ.

    Parameters
    ----------
    delta : float  – target density contrast  Δ ≡ ρ−1
    d     : 2 or 3 – spatial dimension
    sigma2: float  – linear variance entering the one‑loop term
    nu    : float or None –ν parameter in the ellipsoidal map (defaults ν=d)
    tau_left/right : bracketing interval for the root Λ(τ)=δ.

    Returns
    -------
    P_ld : float  – saddle‑point approximation to P(δ).
    """
    Lam, dLam, phi = make_tau_callables(d, sigma2_value=sigma2, nu=nu)

    # ----- Auto-determine bracketing interval if not provided -----
    if tau_left is None or tau_right is None:
        # Expanded search intervals with wider range
        search_intervals = [
            (-5.0, -2.0),
            (-2.0, -1.0),
            (-1.0, -0.5),
            (-0.5, -0.1),
            (-0.1, 0.0),
            (0.0, 0.1),
            (0.1, 0.5),
            (0.5, 1.0),
            (1.0, 2.0),
            (2.0, 5.0),
            (-3.0, 0.0),
            (-1.0, 1.0),
            (-2.0, 2.0),
            (-5.0, 5.0)
        ]
    else:
        search_intervals = [(tau_left, tau_right)]

    # ----- root τ* s.t.  Λ(τ*) = δ -----
    tau_star = None
    for left, right in search_intervals:
        try:
            # Check if the function changes sign over the interval
            f_left = float(Lam(left).real) - delta
            f_right = float(Lam(right).real) - delta
            
            # Debug: print function values for troubleshooting
            if abs(f_left) < 1e-10:  # Very close to zero
                tau_star = left
                break
            if abs(f_right) < 1e-10:  # Very close to zero
                tau_star = right
                break
                
            if f_left * f_right <= 0:  # Sign change indicates root exists
                tau_star = find_root(lambda t: float(Lam(t).real) - delta, left, right)
                break
        except (RuntimeError, ValueError, TypeError) as e:
            continue
    
    if tau_star is None:
        # Try a more systematic grid search as fallback
        tau_grid = [i * 0.1 for i in range(-50, 51)]  # -5.0 to 5.0 in steps of 0.1
        for i in range(len(tau_grid) - 1):
            try:
                left, right = tau_grid[i], tau_grid[i + 1]
                f_left = float(Lam(left).real) - delta
                f_right = float(Lam(right).real) - delta
                if f_left * f_right <= 0:
                    tau_star = find_root(lambda t: float(Lam(t).real) - delta, left, right)
                    break
            except:
                continue
    
    if tau_star is None:
        raise RuntimeError(f"Failed to find τ* for δ={delta}. The function Λ(τ) - {delta} may not have a root in the expanded search range [-5, 5].")

    # ----- prefactor & exponent -----
    pref = sqrt( dLam(tau_star) / (2*pi) )
    exponent = phi(tau_star) - delta*tau_star
    P_ld = pref * exp(exponent)
    return complex(P_ld).real  # PDF is real by construction

# ---------------------------------------------------------------------------
# Function to compute critical tau where dΛ/dτ = 0
# ---------------------------------------------------------------------------


# ---------------------------------------------------------------------------
# example:  d=3, σ²=0.2, ν=3 (Zel'dovich), evaluate at Δ = 0.5
# ---------------------------------------------------------------------------
tau_crit = compute_tau_critical(d=3, nu=3)
print(f"Critical τ = {tau_crit}")

# Check Lambda values around the critical point
Lam, dLam, phi = make_tau_callables(d=3, sigma2_value=0.2, nu=3)
delta_crit = float(Lam(tau_crit).real)
print(f"Λ(τ_crit) = {delta_crit:.6f}")

# Since δ=0.5 might be beyond the critical point, let's check if it's accessible
if 0.5 > delta_crit:
    print(f"Warning: δ=0.5 > δ_crit={delta_crit:.6f}, may not have a solution")
    # Try a smaller delta value that's within range
    test_delta = delta_crit * 0.9
    Pval = pdf_saddle(test_delta, d=3, sigma2=0.2, nu=3)
    print(f"P(δ={test_delta:.3f}) ≈ {Pval:.6g}")
else:
    Pval = pdf_saddle(0.5, d=3, sigma2=0.2, nu=3)
print(f"P(δ=0.5) ≈ {Pval:.6g}")



Critical τ = 0.000000000000000


TypeError: float() argument must be a string or a real number, not 'builtin_function_or_method'